In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import gc
import glob
import os
import cv2
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import imageio as im
import keras
from keras import models
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))

# Any results you write to the current directory are saved as output.


2025-08-18 19:49:47.042447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755546587.055469       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755546587.059458       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 19:49:47.075168: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ImportError: cannot import name 'adam' from 'keras.optimizers' (/usr/local/lib/python3.11/dist-packages/keras/api/optimizers/__init__.py)

In [3]:
# load images dataset
def loadImagesData(glob_path):
    images = []
    names = []
    for img_path in glob.glob(glob_path):
        # load/resize images with cv2
        names.append(os.path.basename(img_path))
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        images.append(img) # already 32x32
    return (images,names)
# map of training label to list of images
trainData = {}
namesData = {}
for label in os.listdir('../input/train/'):
    (images,names) = loadImagesData(f"../input/train/{label}/*.jpg")
    print(f"../input/train/{label}/*.jpg")
    trainData[label] = images
    namesData[label] = names
print("train labels:", ",".join(trainData.keys()))
print(len(trainData['train']))
# show some data
plt.figure(figsize=(4,2))
columns = 4
for i in range(0,8):
    plt.subplot(8 / columns + 1, columns, i + 1)
    plt.imshow(trainData['train'][i])
plt.show()


../input/train/775da0be6da934cb05d6bc7955931dd9.jpg/*.jpg
../input/train/65a52562f1ebce1166d9737ac9d1c0e5.jpg/*.jpg
../input/train/d6243c9afe26f34c5dd8a27bb76210ce.jpg/*.jpg
../input/train/e405a1ed71aec5a0119dfe616252cf35.jpg/*.jpg
../input/train/6a5cd7c3d4e5b917fb37e06270aa1d2e.jpg/*.jpg
../input/train/3c521b66a89ef29344f29889c0ccf6e3.jpg/*.jpg
../input/train/5cfdc3a02cbf7c68b5eaf9a606467a8f.jpg/*.jpg
../input/train/54dd34eeb1d2d7d0c304ff229e0781ff.jpg/*.jpg
../input/train/4c2d70858685be7d59970e7b87049a59.jpg/*.jpg
../input/train/2b35aa2cc4472e298c264e89b96fa17b.jpg/*.jpg
../input/train/54a409762783a325db8e87f70d2604d1.jpg/*.jpg
../input/train/cf4ef20c47177642173f81fdf5f05a2c.jpg/*.jpg
../input/train/85a6b16215ef5ad3e6865e2e80c6ff60.jpg/*.jpg
../input/train/2f9ebd2d0756753d03565f185ee5a7fb.jpg/*.jpg
../input/train/9e5de204cb1cda26f367b217040ad1b8.jpg/*.jpg
../input/train/ddb34bc19d8705b97ef4b7b9d72522bf.jpg/*.jpg
../input/train/9d49a887c2429ff7c7513652db115e63.jpg/*.jpg
../input/train

KeyError: 'train'

In [4]:
train_meta = pd.read_csv('../input/train.csv')
print(train_meta.shape)
print(train_meta.has_cactus.value_counts())
# lookup table of name to has_cactus
lookupY = {}
for i in range(0,len(train_meta)):
    row = train_meta.iloc[i,:]
    lookupY[row.id] = row.has_cactus
train_meta.head()


(14175, 2)
has_cactus
1    10628
0     3547
Name: count, dtype: int64


,id,has_cactus
0,2de8f189f1dce439766637e75df0ee27.jpg,1
1,36704d250f236238e7f996812c48235d.jpg,1
2,eacde22fdc8c175972a5768e3daa8bc9.jpg,1
3,5d442f834da5e57d22b24802c32a8ca8.jpg,1
4,152491e0daf75c0e669400300ff7e645.jpg,1


In [5]:
# build x/y dataset
trainList = []
maxCount = 4364 # number of has_cactus = 0
counts = {'0':0,'1':0}
for (i,image) in enumerate(trainData['train']):
    label = lookupY[namesData['train'][i]]
    counts[str(label)] = 1 + counts[str(label)]
    if counts[str(label)]  maxCount:
        trainList.append({
            'label': label,
            'data': image
        })
# shuffle dataset
random.shuffle(trainList)
# dataframe and display
train_df = pd.DataFrame(trainList)
gc.collect()
print(train_df.shape)
print(train_df.label.value_counts())
train_df.head()


SyntaxError: invalid syntax (242673874.py, line 8)

In [6]:
# encode training data
data_stack = np.stack(train_df['data'].values)
dfloats = data_stack.astype(np.float)
all_x = np.multiply(dfloats, 1.0 / 255.0) # np.array(train_df['data'].values, dtype=np.float) / 255.0
print(all_x.shape)
print(type(all_x))
all_x[0,0,0,0]


NameError: name 'train_df' is not defined

In [7]:
all_y = np.array(train_df.label).astype(np.float)
all_y[0:5]


NameError: name 'train_df' is not defined

In [8]:
# split test/training data
train_x,test_x,train_y,test_y=train_test_split(all_x,all_y,test_size=0.2,random_state=7)
print(train_x.shape,test_x.shape)


NameError: name 'train_test_split' is not defined

In [9]:
# x,y and rotation data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    rotation_range=60,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.2, # zoom images
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images
datagen.fit(train_x)


NameError: name 'ImageDataGenerator' is not defined

In [10]:
# create the network
num_filters = 8
input_shape = train_x.shape[1:]
output_shape = 1
# model
m = Sequential()
def tdsNet(m):
    m.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=input_shape))
    m.add(Conv2D(16, kernel_size=3, activation='relu'))
    m.add(Flatten())
    m.add(Dropout(0.5)) # increases val_acc from 0.89 to 0.92, acc from 0.8932 to 0.8987
#     m.add(BatchNormalization()) # val_acc falling from 0.89 to 0.8322
    m.add(Dense(units = output_shape, activation='sigmoid'))
tdsNet(m)
# compile adam with decay and use binary_crossentropy for single category dataset
m.compile(optimizer = 'nadam',
          loss = 'binary_crossentropy', 
          metrics = ['accuracy'])
# show summary
m.summary()


NameError: name 'train_x' is not defined

In [11]:
# train model
batch_size = 32
history = m.fit_generator(datagen.flow(train_x, train_y,
                          batch_size=batch_size),
                          steps_per_epoch= (train_x.shape[0] // batch_size),
                          epochs = 4,
                          validation_data=(test_x, test_y),
                          workers=4)


NameError: name 'm' is not defined

In [12]:
# create the network
num_filters = 8
input_shape = train_x.shape[1:]
output_shape = 1
# model
m = Sequential()

def cnnNet(m):
    m.add(Conv2D(30, kernel_size=3, activation='relu', input_shape=input_shape))
    m.add(MaxPooling2D(2,2))
    m.add(Conv2D(15, kernel_size=3, activation='relu'))
    m.add(MaxPooling2D(2,2))
    # my update
    '''
    m.add(Conv2D(15, kernel_size=3, activation='relu'))
    m.add(MaxPooling2D(2,2))
    '''
    # my update
    m.add(Dense(7, activation='relu')) # 
    m.add(Flatten())
#     m.add(Dropout(0.8)) # makes no sense: make acc and val_acc lower # increases val_acc from 0.9404 to .., acc from 0.9226 to ..
    m.add(Dense(units = output_shape, activation='sigmoid')) #

'''
# LeNet
def cnnNet(m):
    m.add(Conv2D(20, 5, padding='same', input_shape=input_shape)) # size: 5
    m.add(Activation('relu'))
    m.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    
    m.add(Conv2D(50, 5, padding='same')) # size: 5
    m.add(Activation('relu'))
    m.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    
    m.add(Flatten())
    m.add(Dense(500)) #
    m.add(Activation('relu'))
    
    m.add(Dense(units = output_shape))
    m.add(Activation("sigmoid")) # softmax
'''
    
cnnNet(m)
# compile adam with decay and use binary_crossentropy for single category dataset
m.compile(optimizer = 'nadam',
          loss = 'binary_crossentropy', 
          metrics = ['accuracy'])
# show summary
m.summary()


NameError: name 'train_x' is not defined

In [13]:
### train model
batch_size = 32
history = m.fit_generator(datagen.flow(train_x, train_y,
                          batch_size=batch_size),
                          steps_per_epoch= (train_x.shape[0] // batch_size),
                          epochs = 15, # 4,
                          validation_data=(test_x, test_y),
                          workers=4)


NameError: name 'm' is not defined

In [14]:
# build complete x/y dataset
trainList = []
for (i,image) in enumerate(trainData['train']):
    label = lookupY[namesData['train'][i]]
    trainList.append({
        'label': label,
        'data': image
    })
# shuffle dataset
random.shuffle(trainList)
# dataframe and display
train_df = pd.DataFrame(trainList)
gc.collect()
# encode training data
data_stack = np.stack(train_df['data'].values)
dfloats = data_stack.astype(np.float)
all_x = np.multiply(dfloats, 1.0 / 255.0)
all_x.shape
all_y = np.array(train_df.label).astype(np.float)
# split test/training data
train_x,test_x,train_y,test_y=train_test_split(all_x,all_y,test_size=0.2,random_state=7)
print(train_x.shape,test_x.shape)


KeyError: 'train'

In [15]:
# continue training model
batch_size = 64
history = m.fit_generator(datagen.flow(train_x, train_y,
                          batch_size=batch_size),
                          steps_per_epoch= (train_x.shape[0] // batch_size),
                          epochs = 15, # 4,
                          validation_data=(test_x, test_y),
                          workers=4)


NameError: name 'm' is not defined

In [16]:
# check sample submission format
pd.read_csv('../input/sample_submission.csv').head()


,id,has_cactus
0,09034a34de0e2015a8a28dfe18f423f6.jpg,0.5
1,134f04305c795d6d202502c2ce3578f3.jpg,0.5
2,41fad8d145e6c41868ce3617e30a2545.jpg,0.5
3,35f8a11352c8d41b6231bb33d8d09f7e.jpg,0.5
4,b77dc902b035887cbbc01920ce0e3151.jpg,0.5


In [17]:
# output predicted submission csv
(test_images, test_names) = loadImagesData(f"../input/test/test/*.jpg")
data_stack = np.stack(test_images)
dfloats = data_stack.astype(np.float32)
unknown_x = np.multiply(dfloats, 1.0 / 255.0)
# predict
predicted = np.ravel(m.predict(unknown_x))
submission_df = pd.DataFrame({'id':test_names,'has_cactus':predicted})
submission_df.to_csv('submission.csv', index=False)
len(submission_df)


ValueError: need at least one array to stack